<a href="https://colab.research.google.com/github/Krsnapriya/Krsnapriya/blob/main/Linkedin%20post%20LLMipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install transformers datasets


In [4]:

from google.colab import files
uploaded = files.upload()

Saving linkedin_mid_dataset.csv to linkedin_mid_dataset.csv


In [7]:
import pandas as pd
import io
import re

# A more robust RTF cleaning function (This function is not needed for direct CSV reading)
# def clean_rtf(text):
#     # Remove RTF control words and formatting using a more comprehensive regex
#     text = re.sub(r'\{\*?\\[^{}]+ Weekly updates are essential, highlighting progress and challenges. Communicate proactively with colleagues and stakeholders. Seek feedback and be open to learning and improvement. #teamwork #communication #learninganddevelopment",\}','', text)
#     text = re.sub(r'\\par\s*','\n', text)
#     text = re.sub(r'\\pict.*?\\pard','', text)
#     text = re.sub(r'\{\\field.*?\}','', text)
#     text = re.sub(r'\s+',' ', text)
#     text = re.sub(r'\\rtf[0-9]+\\an','', text)
#     text = re.sub(r'\\.*?\}','', text)
#     text = re.sub(r'\{.*?\}','', text)
#     text = text.replace('\\','')
#     # Remove any remaining curly braces that might interfere with CSV parsing
#     text = text.replace('{', '').replace('}', '')
#     return text.strip()


# Assuming the uploaded file is in the 'uploaded' dictionary from the previous cell
if uploaded:
    # Get the filename and content
    filename = list(uploaded.keys())[0]
    file_content = uploaded[filename]

    # Now, try to read the file content as CSV
    try:
        # Use io.BytesIO to read the binary file content
        df = pd.read_csv(io.BytesIO(file_content))
    except pd.errors.ParserError as e:
        print(f"Attempting to read with different separator or engine due to error: {e}")
        # If the initial read fails, try other common separators or the 'python' engine
        try:
            df = pd.read_csv(io.BytesIO(file_content), sep=';') # Try semicolon as separator
        except:
            try:
                df = pd.read_csv(io.BytesIO(file_content), engine='python') # Try python engine
            except Exception as ex:
                print(f"Could not parse the uploaded file as CSV. Error: {ex}")
                df = None # Set df to None if parsing completely fails

    if df is not None:
        print(f"Successfully loaded data from {filename}")
        display(df.head())
    else:
        print("Failed to load data into DataFrame.")
else:
    print("No file was uploaded.")

Successfully loaded data from linkedin_mid_dataset.csv


,theme,tone,company_or_event,role_or_context,post
0,Startup Launch,Excited,Microsoft,Cloud Architect,"We're launching Microsoft, where I work as Clo..."
1,Hackathon,Proud,Flipkart,UX Designer,Participated in Flipkart's Hackathon and built...
2,Publication,Grateful,TCS,UX Designer,Our paper on climate change just got published...
3,Certification,Confident,Google,Product Manager,Completed Product Manager certification from G...
4,Certification,Inspired,LinkedIn,Cloud Architect,Completed Cloud Architect certification from L...


In [8]:
import pandas as pd

# Load the dataset from the specified CSV file
csv_file_path = "/content/linkedin_mid_dataset.csv"
try:
    df = pd.read_csv(csv_file_path)
    print(f"Successfully loaded data from {csv_file_path}")
    display(df.head())
except FileNotFoundError:
    print(f"Error: The file {csv_file_path} was not found.")
    df = None
except Exception as e:
    print(f"An error occurred while reading the CSV file: {e}")
    df = None

Successfully loaded data from /content/linkedin_mid_dataset.csv


,theme,tone,company_or_event,role_or_context,post
0,Startup Launch,Excited,Microsoft,Cloud Architect,"We're launching Microsoft, where I work as Clo..."
1,Hackathon,Proud,Flipkart,UX Designer,Participated in Flipkart's Hackathon and built...
2,Publication,Grateful,TCS,UX Designer,Our paper on climate change just got published...
3,Certification,Confident,Google,Product Manager,Completed Product Manager certification from G...
4,Certification,Inspired,LinkedIn,Cloud Architect,Completed Cloud Architect certification from L...


In [9]:
from datasets import Dataset
from transformers import GPT2Tokenizer

# Assuming 'df' is your pandas DataFrame loaded correctly
# If df is still empty or incorrect, the previous data loading step needs to be fixed first.
if df is not None and not df.empty:
    dataset = Dataset.from_pandas(df)

    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    tokenizer.pad_token = tokenizer.eos_token

    def preprocess(example):
        input_text = f"[{example['theme']}] [{example['tone']}] {example['company_or_event']} - {example['role_or_context']}: {example['post']}"
        tokens = tokenizer(
            input_text,
            truncation=True,
            padding="max_length",
            max_length=128
        )
        tokens["labels"] = tokens["input_ids"].copy()
        return tokens

    tokenized = dataset.map(preprocess, remove_columns=dataset.column_names)
else:
    print("DataFrame is empty or not loaded correctly. Cannot create dataset for training.")
    tokenized = None # Set tokenized to None if df is not valid

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [10]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

if tokenized is not None:
    model = GPT2LMHeadModel.from_pretrained("gpt2")

    training_args = TrainingArguments(
        output_dir="./linkedin-llm",
        per_device_train_batch_size=4,
        num_train_epochs=3,
        logging_steps=10,
        save_strategy="epoch",
        logging_dir="./logs",
        remove_unused_columns=False   # 🧠 KEY FIX
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized,
        data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
    )

    trainer.train()
else:
    print("Tokenized dataset is not available. Cannot start training.")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: quantumplatores-moon02 (quantumplatores-moon02-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,3.665000
20,2.182000
30,1.490700
40,1.184800
50,0.836200
60,0.686700
70,0.673900
80,0.555000
90,0.574400
100,0.555300


Step,Training Loss
10,3.665000
20,2.182000
30,1.490700
40,1.184800
50,0.836200
60,0.686700
70,0.673900
80,0.555000
90,0.574400
100,0.555300


In [12]:
import nbformat as nbf
from pathlib import Path

# Build a Colab-ready notebook to generate a LinkedIn-style dataset and save as CSV
cells = []

cells.append(nbf.v4.new_markdown_cell("# 📊 Generate LinkedIn Post Dataset (Colab Ready)"))

cells.append(nbf.v4.new_code_cell("""
import pandas as pd
import random

# Define sample categories
themes = ["Job Application", "Hackathon", "University Admission", "Certification", "Startup Launch", "Publication", "Career Shift", "Community Event"]
tones = ["Excited", "Grateful", "Confident", "Proud", "Hopeful", "Joyful", "Inspired"]
companies = ["Google", "Microsoft", "Flipkart", "TCS", "Adobe", "Zomato", "Amazon", "LinkedIn"]
roles = ["Data Scientist", "Software Engineer", "Product Manager", "UX Designer", "Cloud Architect", "ML Engineer"]
topics = ["mental health", "AI ethics", "blockchain voting", "education access", "climate change", "open source"]

# Template-based text generation
templates = {
    "Job Application": [
        "{tone} to apply for the {role} position at {company}.",
        "Just submitted my application to {company} for a {role} role — {tone}!"
    ],
    "Hackathon": [
        "Participated in {company}'s Hackathon and built something awesome in the {role} track!",
        "{tone} to collaborate at {company} Hackathon on a {topic} project."
    ],
    "University Admission": [
        "{tone} to announce my admission to {company} for {role}.",
        "I’ve been accepted into {company} for {role} — feeling so {tone}!"
    ],
    "Certification": [
        "Completed {role} certification from {company}. {tone} for this milestone!",
        "Earned my {role} badge from {company}. It was a {tone} journey."
    ],
    "Startup Launch": [
        "We're launching {company}, where I work as {role}. {tone} to share this with you all!",
        "Introducing our startup: {company} — I'm leading as {role}. {tone} for the future ahead."
    ],
    "Publication": [
        "Our paper on {topic} just got published in {company}. {tone} to share this news!",
        "{tone} to contribute to our publication with {company} as a {role}."
    ],
    "Career Shift": [
        "Excited to shift my career path — now joining {company} as a {role}!",
        "Making the move to {company} as a {role}. Feeling {tone} about this journey."
    ],
    "Community Event": [
        "Volunteered as a {role} at {company} this week. {tone} to be part of it.",
        "Helped organize a {company} event focused on {topic}. {tone} for the turnout!"
    ]
}

# Generate dataset
def generate_dataset(n=1000):
    records = []
    for _ in range(n):
        theme = random.choice(themes)
        tone = random.choice(tones)
        company = random.choice(companies)
        role = random.choice(roles)
        topic = random.choice(topics)
        template = random.choice(templates[theme])

        post = template.format(tone=tone, company=company, role=role, topic=topic)
        records.append({
            "theme": theme,
            "tone": tone,
            "company_or_event": company,
            "role_or_context": role,
            "post": post
        })
    return pd.DataFrame(records)

# Create and preview
df = generate_dataset(1000)
df.head()
"""))

cells.append(nbf.v4.new_markdown_cell("## 💾 Save Dataset to CSV"))

cells.append(nbf.v4.new_code_cell("""
df.to_csv("linkedin_large_dataset.csv", index=False)
print("✅ Saved as linkedin_large_dataset.csv")
"""))

# Save the notebook
nb = nbf.v4.new_notebook(cells=cells)
# Change the save path to the current working directory
notebook_path = Path("generate_linkedin_dataset_colab.ipynb")
notebook_path.write_text(nbf.writes(nb))

notebook_path.name

'generate_linkedin_dataset_colab.ipynb'

Next, upload your data file.

In [13]:
from datasets import Dataset
from transformers import GPT2Tokenizer

# Assuming 'df' is your pandas DataFrame loaded correctly from the previous steps.
# If df is still empty or incorrect, the previous data loading step needs to be fixed first.
if df is not None and not df.empty:
    # Convert the pandas DataFrame to a Hugging Face Dataset
    dataset = Dataset.from_pandas(df)

    # Load the GPT2 tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    tokenizer.pad_token = tokenizer.eos_token  # Set pad token

    # Define the preprocessing function
    def preprocess(example):
        # Format the input text from the DataFrame columns
        input_text = f"[{example['theme']}] [{example['tone']}] {example['company_or_event']} - {example['role_or_context']}: {example['post']}"
        # Tokenize the input text
        tokens = tokenizer(
            input_text,
            truncation=True,
            padding="max_length",
            max_length=128 # You can adjust the max_length as needed
        )
        # Create labels by copying input_ids for language modeling
        tokens["labels"] = tokens["input_ids"].copy()
        return tokens

    # Apply the preprocessing function to the dataset
    tokenized = dataset.map(preprocess, remove_columns=dataset.column_names)

    print("Dataset successfully tokenized and prepared for training.")
    # You can display a sample of the tokenized dataset to verify
    # print(tokenized[0])
else:
    print("DataFrame is empty or not loaded correctly. Cannot create dataset for training.")
    tokenized = None # Set tokenized to None if df is not valid

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset successfully tokenized and prepared for training.


In [ ]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

if tokenized is not None:
    model = GPT2LMHeadModel.from_pretrained("gpt2")

    training_args = TrainingArguments(
        output_dir="./linkedin-llm",
        per_device_train_batch_size=4,
        num_train_epochs=3,
        logging_steps=10,
        save_strategy="epoch",
        logging_dir="./logs",
        remove_unused_columns=False   # 🧠 KEY FIX
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized,
        data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
    )

    trainer.train()
else:
    print("Tokenized dataset is not available. Cannot start training.")

Step,Training Loss
10,3.665000
20,2.182000



Model

In [34]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import ipywidgets as widgets
from IPython.display import display
import torch
from safetensors.torch import load_file
from collections import Counter # Import Counter for repetition reduction


# Load the trained checkpoint
checkpoint_path = "./linkedin-llm/checkpoint-375"

# ✅ Add This Function for Post-Cleanup
from collections import Counter

def reduce_repetition(text):
    words = text.split()
    counts = Counter(words)
    return " ".join([w for w in words if counts[w] <= 3 or len(w) > 6]) # Keep words occurring <= 3 times or longer than 6 chars

def clean_generated_post(prompt, generated_text):
    # Remove the prompt from the output
    cleaned = generated_text.replace(prompt, "").strip()

    # Remove repeated lines
    lines = cleaned.splitlines()
    seen = set()
    deduped_lines = []
    for line in lines:
        # Also check for empty lines before adding to seen set
        if line.strip() not in seen and line.strip() != "":
            deduped_lines.append(line)
            seen.add(line.strip()) # Add stripped line to seen set

    # Clean excessive word repetition
    deduped_text = "\n".join(deduped_lines) # Join with newline to preserve lines
    final_text = reduce_repetition(deduped_text)
    return final_text.strip()


try:
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
    if not hasattr(tokenizer, 'model_max_length') or tokenizer.model_max_length is None:
        tokenizer.model_max_length = 128

    # Load the base model architecture
    model = AutoModelForCausalLM.from_pretrained("gpt2")

    # Load the state dictionary from the checkpoint using safetensors
    state_dict = load_file(f"{checkpoint_path}/model.safetensors")

    # Load the state dictionary into the model, allowing for missing keys
    # We are keeping strict=False as the previous error indicated missing keys
    model.load_state_dict(state_dict, strict=False)

    # Initialize generator
    generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

    # Input UI
    theme_input = widgets.Dropdown(
        options=[
            "Job Application", "Hackathon", "University Admission",
            "Certification", "Startup Launch", "Publication",
            "Career Shift", "Community Event"
        ],
        description="Theme:"
    )
    tone_input = widgets.Dropdown(
        options=["Excited", "Grateful", "Confident", "Proud", "Hopeful", "Joyful", "Inspired"],
        description="Tone:"
    )
    company_input = widgets.Text(description="Company:")
    role_input = widgets.Text(description="Role:")
    generate_button = widgets.Button(description="Generate Post")
    output_area = widgets.Output()

    display(theme_input, tone_input, company_input, role_input, generate_button, output_area)

    def on_generate_button_clicked(b):
        with output_area:
            output_area.clear_output()
            theme = theme_input.value
            tone = tone_input.value
            company = company_input.value
            role = role_input.value

            # ✅ 1. Add Better Prompt Formatting
            prompt = f"[{theme}] [{tone}] {company} - {role}:\n"

            try:
                # ✅ 2. Limit Output and Clean Redundancy
                result = generator(
                    prompt,
                    max_new_tokens=80,
                    num_return_sequences=1,
                    do_sample=True,
                    top_p=0.95,
                    temperature=0.9,
                    eos_token_id=tokenizer.eos_token_id # Add end of sequence token
                )

                # 🔁 Use It Like This in Your Generation Block
                raw_output = result[0]["generated_text"]
                final_post = clean_generated_post(prompt, raw_output)


                print("📣 Generated LinkedIn Post:\n")
                print(final_post)
            except Exception as e:
                print(f"❌ Error during generation: {e}")

    generate_button.on_click(on_generate_button_clicked)

except Exception as e:
    print(f"❌ Error loading model or setting up UI: {e}")

Device set to use cpu


Dropdown(description='Theme:', options=('Job Application', 'Hackathon', 'University Admission', 'Certification…

Dropdown(description='Tone:', options=('Excited', 'Grateful', 'Confident', 'Proud', 'Hopeful', 'Joyful', 'Insp…

Text(value='', description='Company:')

Text(value='', description='Role:')

Button(description='Generate Post', style=ButtonStyle())

Output()

In [16]:
import os
print(os.listdir("./linkedin-llm"))

['checkpoint-250', 'checkpoint-125', 'checkpoint-375']


In [24]:
import os
print(os.listdir("./linkedin-llm/checkpoint-375"))

['merges.txt', 'rng_state.pth', 'vocab.json', 'scheduler.pt', 'model.safetensors', 'config.json', 'generation_config.json', 'optimizer.pt', 'special_tokens_map.json', 'trainer_state.json', 'tokenizer_config.json', 'training_args.bin']
